# **Medical Report Summerization**
- In this project i will make a medical report summerizer.
- The goal of this project is to summerize the user medical report.

# Load Dataset

In [ ]:
!kaggle datasets download -d tboyle10/medicaltranscriptions

Dataset URL: https://www.kaggle.com/datasets/tboyle10/medicaltranscriptions
License(s): CC0-1.0
 83% 4.00M/4.85M [00:01<00:00, 4.30MB/s]
100% 4.85M/4.85M [00:01<00:00, 3.66MB/s]


In [ ]:
!unzip /content/medicaltranscriptions.zip

Archive:  /content/medicaltranscriptions.zip
  inflating: mtsamples.csv           


# Import Packages

In [1]:
!pip  install --upgrade transformers pandas numpy tensorflow datasets torch bitsandbytes sentencepiece accelerate peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 4.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadat

In [2]:
!pip show transformers pandas numpy tensorflow datasets torch bitsandbytes sentencepiece accelerate peft

Name: transformers
Version: 4.44.1
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft
---
Name: pandas
Version: 2.2.2
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: 
Author-email: The Pandas Development Team <pandas-dev@python.org>
License: BSD 3-Clause License
        
        Copyright (c) 2008-2011, AQR Capital Management, LLC, Lambda Foundry, Inc. and PyData Development Team
        All rights reserved.
        
        Copyright (c) 

In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
# import datasets
from datasets import Dataset

# Read the Data

In [ ]:
df=pd.read_csv('/content/mtsamples.csv')

In [ ]:
df.head(2)

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."


# Observation
- In this Dataset there are many columns but we can get only `description` and `transcription` columns b/c we can do medical report summerization task

In [ ]:
df.isna().sum()

,0
Unnamed: 0,0
description,0
medical_specialty,0
sample_name,0
transcription,33
keywords,1068


In [ ]:
df.shape

(4999, 6)

In [ ]:
df=df.drop(columns=['Unnamed: 0','keywords']).dropna()

In [ ]:
df.isna().sum()

,0
description,0
medical_specialty,0
sample_name,0
transcription,0


In [ ]:
df.head(2)

,description,medical_specialty,sample_name,transcription
0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr..."
1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb..."


In [ ]:
"sample_name"+df["sample_name"][1] +" ,"+"Medicl Speciality"+df["medical_specialty"][1] +"," +df['transcription'][1]

'sample_name Laparoscopic Gastric Bypass Consult - 2  ,Medicl Speciality Bariatrics,PAST MEDICAL HISTORY:, He has difficulty climbing stairs, difficulty with airline seats, tying shoes, used to public seating, and lifting objects off the floor.  He exercises three times a week at home and does cardio.  He has difficulty walking two blocks or five flights of stairs.  Difficulty with snoring.  He has muscle and joint pains including knee pain, back pain, foot and ankle pain, and swelling.  He has gastroesophageal reflux disease.,PAST SURGICAL HISTORY:, Includes reconstructive surgery on his right hand 13 years ago.  ,SOCIAL HISTORY:, He is currently single.  He has about ten drinks a year.  He had smoked significantly up until several months ago.  He now smokes less than three cigarettes a day.,FAMILY HISTORY:, Heart disease in both grandfathers, grandmother with stroke, and a grandmother with diabetes.  Denies obesity and hypertension in other family members.,CURRENT MEDICATIONS:, None.

# Onservation
- Now we can do this thing we can add the other columns in the transation columns

In [ ]:
df.head(1)

,description,medical_specialty,sample_name,transcription
0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr..."


In [ ]:
df['transcription']= "sample_name: " + df["sample_name"] +"," + "medical_specialty: " + df["medical_specialty"] +"," + df['transcription']

# Now we can drop the columns except `description` and `transation`

In [ ]:
df.drop(columns=['medical_specialty','sample_name'],inplace=True)

In [ ]:
df['description'][100]

' Right inguinal hernia.    Right direct inguinal hernia repair with PHS mesh system.  The Right groin and abdomen were prepped and draped in the standard sterile surgical fashion.  An incision was made approximately 1 fingerbreadth above the pubic tubercle and in a skin crease. '

In [ ]:
df['transcription'][100]

"sample_name:  Inguinal Hernia Repair - 4 ,medical_specialty:  Urology,PREOPERATIVE DIAGNOSIS: , Right inguinal hernia. ,POSTOPERATIVE DIAGNOSIS:,  Right direct inguinal hernia. ,PROCEDURE:,  Right direct inguinal hernia repair with PHS mesh system. ,ANESTHESIA:,  General with endotracheal intubation. ,PROCEDURE IN DETAIL: , The patient was taken to the operating room and placed supine on the operating table.  General anesthesia was administered with endotracheal intubation.  The Right groin and abdomen were prepped and draped in the standard sterile surgical fashion.  An incision was made approximately 1 fingerbreadth above the pubic tubercle and in a skin crease.  Dissection was taken down through the skin and subcutaneous tissue.  Scarpa's fascia was divided, and the external ring was located.  The external oblique was divided from the external ring up towards the anterior superior iliac spine.  The cord structures were then encircled.  Careful inspection of the cord structures did 

# Important
- Now we can convert the pandas dataframe to hugging facw data format

# Some important thing

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df.drop(columns=['index'],inplace=True)

In [ ]:
df.head(2)

,description,transcription
0,A 23-year-old white female presents with comp...,"sample_name: Allergic Rhinitis ,medical_speci..."
1,Consult for laparoscopic gastric bypass.,sample_name: Laparoscopic Gastric Bypass Cons...


# End of important thing

In [ ]:
data=Dataset.from_pandas(df)

In [ ]:
data

Dataset({
    features: ['description', 'transcription'],
    num_rows: 4966
})

# Observation
- we can drop the `Index_level` column

In [ ]:
# data=data.remove_columns(['__index_level_0__'])

In [ ]:
data

Dataset({
    features: ['description', 'transcription'],
    num_rows: 4966
})

# Split the data into train and test set

In [ ]:
data=data.train_test_split(test_size=0.2)

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['description', 'transcription'],
        num_rows: 3972
    })
    test: Dataset({
        features: ['description', 'transcription'],
        num_rows: 994
    })
})

# Saperate the train and test set

In [ ]:
train_data=data['train']

In [ ]:
test_data=data['test']

In [ ]:
train_data.features

{'description': Value(dtype='string', id=None),
 'transcription': Value(dtype='string', id=None)}

In [ ]:
test_data.features

{'description': Value(dtype='string', id=None),
 'transcription': Value(dtype='string', id=None)}

# Get the 1 record of train data

In [ ]:
print("Description: ",train_data[0]['description'])
print("*"*len(train_data[0]['description']))
print()

print("Transcption: ",train_data[0]['transcription'])

Description:   Pyogenic granuloma, left lateral thigh.   Excision of recurrent pyogenic granuloma.
************************************************************************************

Transcption:  sample_name:  Granuloma Excision ,medical_specialty:  Surgery,PREOPERATIVE DIAGNOSIS: , Pyogenic granuloma, left lateral thigh.,POSTOPERATIVE DIAGNOSIS: , Pyogenic granuloma, left lateral thigh.,ANESTHESIA:,  General.,PROCEDURE: , Excision of recurrent pyogenic granuloma.,INDICATIONS: , The patient is 12-year-old young lady, who has a hand-sized congenital vascular malformation on her left lateral thigh below the greater trochanter, which was described by her parents as a birthmark.  This congenital cutaneous vascular malformation faded substantially over the first years of her life and has regressed to a flat, slightly hyperpigmented lesion.  Although no isolated injury event can be recalled, the patient has developed a pyogenic granuloma next to the distal portion of this lesion on her mi

# Now we can Load the `Tokenizer` and `Model`

In [ ]:
# !pip install transformers[torch]

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

# Test the `tokenizer`

In [ ]:
text_token=tokenizer.tokenize("Consult for laparoscopic gastric bypass")
print("Text Token:")
print(text_token)


text_token_id=tokenizer.convert_tokens_to_ids(text_token)
print("Token Text Id: ")
print(text_token_id)

print("Convert the token into original text: ")
original_token=tokenizer.convert_ids_to_tokens(text_token_id)
print(original_token)

Text Token:
['Cons', 'ult', 'Ġfor', 'Ġlap', 'ar', 'oscopic', 'Ġgast', 'ric', 'Ġbypass']
Token Text Id: 
[24514, 6070, 13, 8040, 271, 40229, 26394, 4063, 15037]
Convert the token into original text: 
['Cons', 'ult', 'Ġfor', 'Ġlap', 'ar', 'oscopic', 'Ġgast', 'ric', 'Ġbypass']


# Other way of tokenize text

In [ ]:
tokenizer("Consult for laparoscopic gastric bypass.",padding='max_length',max_length=1000,truncation=True,return_tensors="pt")

{'input_ids': tensor([[    0, 24514,  6070,    13,  8040,   271, 40229, 26394,  4063, 15037,
             4,     2,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,  

# Make a Function for tokenize the text

In [ ]:
def tokenize_fun(data):
  model_input=tokenizer(data['transcription'],
                        padding='max_length',truncation=True,
                        max_length=500,return_tensors="pt")

  label=tokenizer(data['description'],
                        padding='max_length',truncation=True,
                        max_length=150,return_tensors="pt")

  return {
      "input_ids":model_input['input_ids'],
      "attention_mask":model_input['attention_mask'],
      "labels":label['input_ids']
  }

# Test the fun

In [ ]:
print(tokenize_fun(train_data[0]))

{'input_ids': tensor([[    0, 14029,  1215, 13650,    35,  1437,  9328,   922,  4982,  3015,
         37938,  2156, 30885,  1215, 19423,  2553,    35,  1437, 26793,     6,
         43869, 35918, 39869,   211,  2889, 17567,  3196,  1729,    35,  2156,
         19972, 23982, 17227,   922,  4982,     6,   314, 30972, 18781,   482,
         47060, 35918, 39869,   211,  2889, 17567,  3196,  1729,    35,  2156,
         19972, 23982, 17227,   922,  4982,     6,   314, 30972, 18781,   482,
          1889,  1723,  3732,  1723,  2889, 46686,  1437,  1292,   482,  4454,
          4571,  1691, 12435,    35,  2156,  3015, 37938,     9, 35583, 19290,
         23982, 17227,   922,  4982,   482, 13796,  2371, 14939,    35,  2156,
            20,  3186,    16,   316,    12,   180,    12,   279,   664,  6429,
             6,    54,    34,    10,   865,    12,  8407, 36764,  8632, 37256,
          8196, 26941,    15,    69,   314, 30972, 18781,   874,     5,  2388,
         14168, 14717,  1334,     6,  

# Apply the fun

In [ ]:
print("Mapping train data")
tokenize_train_data=train_data.map(tokenize_fun,batched=True)

print("Mapping test data")
tokenize_test_data=test_data.map(tokenize_fun,batched=True)


Mapping train data


Map:   0%|          | 0/3972 [00:00<?, ? examples/s]

Mapping test data


Map:   0%|          | 0/994 [00:00<?, ? examples/s]

In [ ]:
tokenize_train_data

Dataset({
    features: ['description', 'transcription', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 3972
})

# Observation
- we can get only `input_ids` ,`attention_mask` and `labels` columns only

In [ ]:
tokenize_train_data=tokenize_train_data.remove_columns(['description','transcription'])

In [ ]:
tokenize_train_data

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3972
})

In [ ]:
tokenize_test_data=tokenize_test_data.remove_columns(['description','transcription'])

In [ ]:
tokenize_test_data

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 994
})

# load the model

In [ ]:
# from transformers import BitsAndBytesConfig

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

# Apply Lora Config

In [ ]:
# from peft import LoraConfig, get_peft_model, TaskType

# Now we can make a Training Args

In [ ]:

# lora_config = LoraConfig(
#     task_type=TaskType.SEQ_2_SEQ_LM,  # Set the task type to sequence-to-sequence language modeling
#     r=4,  # Low-rank dimension
#     lora_alpha=8,  # Scaling factor
#     lora_dropout=0.5,  # Dropout rate for LoRA
#     target_modules=["encoder.layers.0.self_attn.q_proj",
#                      "encoder.layers.0.self_attn.k_proj",
#                      "encoder.layers.0.self_attn.v_proj",
#                      "encoder.layers.0.fc1"],  # Target modules for LoRA
# )

In [ ]:
# peft_model = get_peft_model(model, lora_config)

# Training Args

In [ ]:
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
train_args=Seq2SeqTrainingArguments(
    output_dir=".model",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    num_train_epochs=5,
    save_steps=500,
    save_total_limit=3,
    gradient_accumulation_steps=8,
)

# Make a `DataColator`

In [ ]:
data_colator=DataCollatorForSeq2Seq(tokenizer,model=model,return_tensors="pt")

# Make a `training` API

In [ ]:
# model.resize_token_embeddings(len(tokenizer))

In [ ]:
trainer=Seq2SeqTrainer(
    model=model,
    args=train_args,
    data_collator=data_colator,
    train_dataset=tokenize_train_data,
    eval_dataset=tokenize_test_data
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
0,No log,0.117769
1,No log,0.089681


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# df.to_csv('/content/drive/MyDrive/medical_report_summary/data.csv',index=False)

# Save the model in google drive

In [ ]:
trainer.model.save_pretrained("/content/drive/MyDrive/medical_report_summary/model")
tokenizer.save_pretrained("/content/drive/MyDrive/medical_report_summary/tokenizer")

('/content/drive/MyDrive/medical_report_summary/tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/medical_report_summary/tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/medical_report_summary/tokenizer/spiece.model',
 '/content/drive/MyDrive/medical_report_summary/tokenizer/added_tokens.json')

# Evulation

In [ ]:
trainer.evaluate()

{'eval_loss': 0.12857800722122192,
 'eval_runtime': 48.3533,
 'eval_samples_per_second': 20.557,
 'eval_steps_per_second': 5.15,
 'epoch': 2.996978851963746}

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=447a1f90867235b3c526ed1dbff8739bc6deaf327008084bfe763b9b78bc0086
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
from rouge_score import rouge_scorer

In [ ]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

def calculate_rouge(model, tokenizer, dataset):
    predictions = []
    references = []

    for example in dataset:
        input_text = example['transcription']
        reference_summary = example['description']

        inputs = tokenizer(input_text, max_length=300, truncation=True, padding='max_length', return_tensors='pt').to('cuda')
        summary_ids = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=150)
        predicted_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

        predictions.append(predicted_summary)
        references.append(reference_summary)

    # Calculate ROUGE scores
    all_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
    for ref, pred in zip(references, predictions):
        scores = scorer.score(ref, pred)
        for key in all_scores:
            all_scores[key].append(scores[key].fmeasure)

    # Average scores
    average_scores = {key: sum(values) / len(values) for key, values in all_scores.items()}
    return average_scores

In [ ]:
# Calculate ROUGE scores for the test set
rouge_scores = calculate_rouge(model, tokenizer, test_data)

# Print the results
print(rouge_scores)

In [4]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer

config = PeftConfig.from_pretrained("sami606713/medical_report_summarizer")
base_model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")
model = PeftModel.from_pretrained(base_model, "sami606713/medical_report_summarizer")
tokenizer = AutoTokenizer.from_pretrained("sami606713/medical_report_summarizer")

# New data to test on
new_data = [
    "sample_name: John Doe, medical_specialty: Cardiology, Patient presents with chest pain and shortness of breath.",
    "sample_name: Jane Smith, medical_specialty: Orthopedics, Patient reports pain in the right knee after a fall."
]

# Generate summaries for the new data
for input_text in new_data:
    inputs = tokenizer(input_text, max_length=512, truncation=True, padding='max_length', return_tensors='pt')
    summary_ids = model.generate(**inputs, max_length=300)
    predicted_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    print(f"Input: {input_text}\nSummary: {predicted_summary}\n")


tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

Input: sample_name: John Doe, medical_specialty: Cardiology, Patient presents with chest pain and shortness of breath.
Summary: sample_name: John Doe, medical_specialty: Cardiology, Patient presents with chest pain and shortness of breath. Patient's name is John Doe and he has been in the hospital for a few days. He has been diagnosed with a chest infection and is being treated with medication.

Input: sample_name: Jane Smith, medical_specialty: Orthopedics, Patient reports pain in the right knee after a fall.
Summary: sample_name: Jane Smith, medical_specialty: Orthopedics. Patient reports pain in the right knee after a fall. Patient is given an epidural injection to treat the pain in her right knee. Patients are given an injection to help with the pain.



In [5]:
test_report ="""
**Patient:** Mr. George Thompson
**Age:** 67 years
**Gender:** Male
**Date:** August 10, 2024

**Chief Complaint:**
Complaints of fatigue, palpitations, and dizziness for the past 2 weeks.

**History of Present Illness:**
Mr. Thompson, a 67-year-old male, presents with a history of progressive fatigue, palpitations, and occasional dizziness over the past two weeks. He reports that the palpitations are irregular and occur several times a day, often lasting a few minutes. There is no associated chest pain, but he feels lightheaded during these episodes. He denies shortness of breath or syncope.

**Past Medical History:**
- Coronary Artery Disease (diagnosed 5 years ago)
- Atrial Fibrillation (diagnosed 3 years ago)
- Hypercholesterolemia

**Medications:**
- Warfarin 5 mg daily
- Metoprolol 50 mg twice daily
- Atorvastatin 40 mg daily

**Allergies:**
- No known drug allergies

**Family History:**
- Father: Deceased at 70 (coronary artery disease)
- Mother: Hypertension

**Social History:**
- Former smoker: 1 pack per day for 40 years, quit 10 years ago
- Alcohol: Occasional
- Retired lawyer

**Physical Examination:**
- **Blood Pressure:** 130/80 mmHg
- **Heart Rate:** Irregularly irregular at 85 beats per minute
- **Respiratory Rate:** 16 breaths per minute
- **Temperature:** 98.7°F (37.1°C)
- **Oxygen Saturation:** 97% on room air

**Assessment/Plan:**
- Continue current medications
- Consider Holter monitor for further evaluation of palpitations
- Follow up in 2 weeks for further evaluation.
"""

In [3]:
test_report=test_report.replace("\n"," ")

In [6]:
inputs = tokenizer(test_report, max_length=1000, truncation=True, padding='max_length', return_tensors='pt')
summary_ids = model.generate(**inputs, max_length=300)
predicted_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(f"Input: {test_report}\nSummary: {predicted_summary}\n")

Input: 
**Patient:** Mr. George Thompson  
**Age:** 67 years  
**Gender:** Male  
**Date:** August 10, 2024

**Chief Complaint:**  
Complaints of fatigue, palpitations, and dizziness for the past 2 weeks.

**History of Present Illness:**  
Mr. Thompson, a 67-year-old male, presents with a history of progressive fatigue, palpitations, and occasional dizziness over the past two weeks. He reports that the palpitations are irregular and occur several times a day, often lasting a few minutes. There is no associated chest pain, but he feels lightheaded during these episodes. He denies shortness of breath or syncope.

**Past Medical History:**  
- Coronary Artery Disease (diagnosed 5 years ago)  
- Atrial Fibrillation (diagnosed 3 years ago)  
- Hypercholesterolemia  

**Medications:**  
- Warfarin 5 mg daily  
- Metoprolol 50 mg twice daily  
- Atorvastatin 40 mg daily  

**Allergies:**  
- No known drug allergies  

**Family History:**  
- Father: Deceased at 70 (coronary artery disease)  


# Split the text into Chunks

In [36]:
def split_to_chunk(text, max_len=300, overlap=20):
    words = text.split()  # Split the text into words
    chunks = []

    if len(words) <= max_len:
        chunks.append(text)
    else:
        start = 0
        while start < len(words):
            # Calculate the end position for words
            end = min(start + max_len, len(words))

            # Append the current chunk (join words back into a string)
            chunks.append(" ".join(words[start:end]))

            # Update the start position for the next chunk, accounting for overlap
            start += max_len - overlap

    return chunks

In [46]:
try:
  text_chunk=split_to_chunk(test_report)
except Exception as e:
  print(e)

In [50]:
final_summary=[]
for curr_text in text_chunk:
  # Tokenize the curr_text
  # print(curr_text)
  curr_input=tokenizer(curr_text,max_length=1000,truncation=True,padding='max_length',return_tensors='pt')
  # generate the summary
  curr_eummary_idx=model.generate(**curr_input, max_length=150)
  # Decode the summary
  predicted_summary = tokenizer.decode(curr_eummary_idx[0], skip_special_tokens=True)
  print("Summary:\n",predicted_summary)
  print("*"*100)
  final_summary.append(predicted_summary)

Summary:
 Mr. George Thompson, a 67-year-old male, presents with a history of progressive fatigue, palpitations, and occasional dizziness over the past two weeks. He reports that thepalpitations are irregular and occur several times a day, often lasting a few minutes. There is no associated chest pain, but he feels lightheaded during these episodes.
****************************************************************************************************


In [51]:
# test_report

In [52]:
" ".join(final_summary)

'Mr. George Thompson, a 67-year-old male, presents with a history of progressive fatigue, palpitations, and occasional dizziness over the past two weeks. He reports that thepalpitations are irregular and occur several times a day, often lasting a few minutes. There is no associated chest pain, but he feels lightheaded during these episodes.'

In [45]:
len(test_report.split())

225

In [7]:
import torch
print(torch.__version__)

2.4.0+cu121
